<h1 align ="center"> REST API In Context Learning Samples</h1>
<hr>
   
# Chat Completions

In [7]:
import json
import os
import base64
%run shared_functions.ipynb

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup search_service_endpoint, search_index_name, and search_query_key

In [8]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the Azure Search service endpoint.e.g. https://<your search service name>.search.windows.net  
search_service_endpoint = config_details['AZURE_SEARCH_SERVICE_ENDPOINT']

# Setting up the Azure Search service index
search_index_name = config_details['AZURE_SEARCH_INDEX_NAME']

# Setting up the Azure Search service query key
search_query_key = os.getenv("AZURE_SEARCH_QUERY_KEY")

### Create Azure Search Index

In [10]:
# Using the Azure Search service create the index with image embeddings
#https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/azure-search-vector-image-index-creation-python-sample.ipynb

### Call GPT-4V API with Image

In [9]:
# System messages and user prompt
sys_message = "You are an AI assistant that helps people find information."
user_prompt = "What are the types of the apple(s) shown in this image?"

# Encode the image in base64
image_file_path = "../../common/images/test_Gala.jpeg"  # Another example including two apples: "../../common/images/two_apples.jpeg".
with open(image_file_path, 'rb') as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
  
messages = [
    {
      "role": "system",
      "content": [
          {
            "type": "text", 
            "text": sys_message
          }
      ]
    },
    {
      "role": "user",
      "content": [
          {
            "type": "text",
            "text": user_prompt # Prompt for the user
          },
          {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}" # Image to be processed
          }
          }
      ]
    }
  ]

in_context_config = {
    'endpoint': search_service_endpoint,
    'key': search_query_key,
    'indexName': search_index_name
}    

try:
    response_content = call_GPT4V_image(messages, in_context=in_context_config)
    print(response_content['choices'][0]['message']['content'])  # Print the content of the response
except Exception as e:
    raise SystemExit(f"Failed to call GPT-4V API. Error: {e}")

Gala
